In [1]:
import torch
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration
tr = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
mdl = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
torch_device = 'cpu'


In [3]:
def bart_summarize(text, num_beams=20, length_penalty=2, max_length=2048, min_length=56, no_repeat_ngram_size=2):

      text = text.replace('\n','')
      text_input_ids = tr.batch_encode_plus([text], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
      summary_ids = mdl.generate(text_input_ids, num_beams=int(num_beams), length_penalty=float(length_penalty), max_length=int(max_length), min_length=int(min_length), no_repeat_ngram_size=int(no_repeat_ngram_size))
      summary_txt = tr.decode(summary_ids.squeeze(), skip_special_tokens=True)
      return summary_txt

In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl
from sentence_transformers import SentenceTransformer, util
import torch
#import os


df = pd.read_csv('combined_paris.csv')


df_combined = df.sort_values(['Hotel']).groupby('Hotel', sort=False).review.apply(''.join).reset_index(name='all_review')

import re

df_combined['all_review'] = df_combined['all_review'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
def lower_case(input_str):
        input_str = input_str.lower()
        return input_str

In [5]:
df_combined['all_review']= df_combined['all_review'].apply(lambda x: lower_case(x))

df = df_combined

df_sentences = df_combined.set_index("all_review")

df_sentences = df_sentences["Hotel"].to_dict()
df_sentences_list = list(df_sentences.keys())


In [6]:
df_combined.head().T

,0,1,2,3,4
Hotel,25hours Hotel Terminus Nord,Acacias Etoile Hotel,COQ Hotel Paris,Campanile Paris 14 - Maine Montparnasse,Cler Hotel
all_review,weve spent lots of time in paris and this was ...,the hotel is great for value the breakfast sel...,stayed for a short city break the hotel is a ...,room was very clean transportation is very ne...,we had the best stay at cler hotel the locati...


In [7]:
long_summary = []

for i in range(len(df_combined)):
    t = bart_summarize(df_combined['all_review'][i])
    long_summary.append(t)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [8]:
df_combined['summary'] = long_summary

In [9]:
df_combined.to_csv('df_combined_paris.csv',index=False)

In [10]:
df_combined.head()

,Hotel,all_review,summary
0,25hours Hotel Terminus Nord,weve spent lots of time in paris and this was ...,we were blown away by this excellent hotel we ...
1,Acacias Etoile Hotel,the hotel is great for value the breakfast sel...,The hotel is great for value the breakfast sel...
2,COQ Hotel Paris,stayed for a short city break the hotel is a ...,stayed for a short city break the hotel is a ...
3,Campanile Paris 14 - Maine Montparnasse,room was very clean transportation is very ne...,hotel turned out to be perfect for our short ...
4,Cler Hotel,we had the best stay at cler hotel the locati...,we had the best stay at cler hotel the locati...


In [3]:
!ls

Dockerfile                            df_combined.csv
Hotel New York Combined.csv           en_core_web_sm-3.2.0-py3-none-any.whl
README.md                             query_generator.ipynb
Untitled.ipynb                        requirements.txt
app.py                                summary.ipynb
app.yaml


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl
from sentence_transformers import SentenceTransformer, util
import torch
df_combined_paris_summary = pd.read_csv('df_combined_paris.csv')

/Users/aimzlicious/miniforge3/envs/tf_m1/lib/python3.8/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [4]:
df_combined_paris.head()

,Hotel,all_review,summary
0,25hours Hotel Terminus Nord,weve spent lots of time in paris and this was ...,we were blown away by this excellent hotel we ...
1,Acacias Etoile Hotel,the hotel is great for value the breakfast sel...,The hotel is great for value the breakfast sel...
2,COQ Hotel Paris,stayed for a short city break the hotel is a ...,stayed for a short city break the hotel is a ...
3,Campanile Paris 14 - Maine Montparnasse,room was very clean transportation is very ne...,hotel turned out to be perfect for our short ...
4,Cler Hotel,we had the best stay at cler hotel the locati...,we had the best stay at cler hotel the locati...


In [5]:
df_paris = pd.read_csv('paris_clean_newer.csv')

In [9]:
hotel=pd.DataFrame(df_paris['Hotel'].drop_duplicates())

In [11]:
df_combined_paris.merge(hotel,how='left')

,Hotel,all_review,summary
0,25hours Hotel Terminus Nord,weve spent lots of time in paris and this was ...,we were blown away by this excellent hotel we ...
1,Acacias Etoile Hotel,the hotel is great for value the breakfast sel...,The hotel is great for value the breakfast sel...
2,COQ Hotel Paris,stayed for a short city break the hotel is a ...,stayed for a short city break the hotel is a ...
3,Campanile Paris 14 - Maine Montparnasse,room was very clean transportation is very ne...,hotel turned out to be perfect for our short ...
4,Cler Hotel,we had the best stay at cler hotel the locati...,we had the best stay at cler hotel the locati...
...,...,...,...
89,Sofitel Paris Le Faubourg,4 years ago i was the last time at sofitel le ...,4 years ago i was the last time at sofitel le ...
90,St Christopher's Gare du Nord Paris,when arriving to the area it felt a little dan...,Barry is the best bartender in paris cheers gr...
91,St Christopher's Inn Canal Paris,ive stayed at st christopher inn canal in pari...,ive stayed at st christopher inn canal in pari...
92,Touring Hotel,hotel is in a great location minutes walk fro...,Hotel is in a great location minutes walk fro...
